In [1]:
%pip install onnx onnxscript

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from google.colab import files


# 1. Configuración y Carga de Datos (Batch size ayuda a la velocidad)
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # Normalización estándar de MNIST
])

# Descarga y carga automática (mucho más limpio que fetch_openml a mano)
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=0)

# 2. Definir la Arquitectura (CNN Simple)
# Esto reemplaza a HOG. La red aprende a buscar bordes y formas.
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Capa convolucional: Extrae características (como HOG pero automático)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Capas lineales: Clasificación (como tu SVM/DecisionTree)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10) # 10 dígitos de salida

    def forward(self, x):
        # x es la imagen cruda [Batch, 1, 28, 28]
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2) # Reduce tamaño a 14x14
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2) # Reduce tamaño a 7x7
        
        x = torch.flatten(x, 1) # Aplanar para la parte densa
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x # Devuelve logits (sin softmax, CrossEntropyLoss lo maneja)

# 3. Entrenamiento (Train loop)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(f"Entrenando en: {device}")

def train(epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print(f'Epoca {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.4f}')

# 4. Evaluación
def test():
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / len(test_loader.dataset)
    print(f'\nPrecision en Test set: {acc:.2f}%')

# Ejecutar
train(epochs=2) # Con 2 épocas basta para >98%
test()

# --- PASO CRUCIAL PARA MLOPS: EXPORTAR A ONNX ---
print("\nExportando a ONNX...")
dummy_input = torch.randn(1, 1, 28, 28, device=device) # Un ejemplo de entrada falsa
torch.onnx.export(model, 
                  dummy_input, 
                  "mnist_cnn.onnx", 
                  input_names=['input'], 
                  output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Modelo guardado como 'mnist_cnn.onnx'")

files.download('mnist_cnn.onnx')

Entrenando en: cuda
Epoca 1 [0/60000] Loss: 2.2935
Epoca 1 [6400/60000] Loss: 0.2782
Epoca 1 [12800/60000] Loss: 0.1545
Epoca 1 [19200/60000] Loss: 0.0409
Epoca 1 [25600/60000] Loss: 0.0769
Epoca 1 [32000/60000] Loss: 0.0504
Epoca 1 [38400/60000] Loss: 0.0101
Epoca 1 [44800/60000] Loss: 0.0078
Epoca 1 [51200/60000] Loss: 0.0198
Epoca 1 [57600/60000] Loss: 0.0559
Epoca 2 [0/60000] Loss: 0.0657
Epoca 2 [6400/60000] Loss: 0.0812
Epoca 2 [12800/60000] Loss: 0.0696
Epoca 2 [19200/60000] Loss: 0.1320
Epoca 2 [25600/60000] Loss: 0.1256
Epoca 2 [32000/60000] Loss: 0.0069
Epoca 2 [38400/60000] Loss: 0.0911
Epoca 2 [44800/60000] Loss: 0.0182
Epoca 2 [51200/60000] Loss: 0.0150
Epoca 2 [57600/60000] Loss: 0.0405

Precision en Test set: 98.82%

Exportando a ONNX...


/tmp/ipython-input-3423498066.py:93: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(model,


[torch.onnx] Obtain model graph for `SimpleCNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleCNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 1 of general pattern rewrite rules.
Modelo guardado como 'mnist_cnn.onnx'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>